In [ ]:
# | default_exp testing.client

In [ ]:
# | export

from collections import namedtuple
from unittest.mock import AsyncMock, MagicMock
from typing import *
import functools

from fastcore.foundation import patch

from fastkafka.application import FastKafka

In [ ]:
import pytest

from pydantic import BaseModel, Field


from fastkafka.helpers import create_missing_topics
from fastkafka.testing import LocalKafkaBroker

In [ ]:
# | export

@patch
def create_mocks(self: FastKafka) -> None:
    app_methods = [f for f, _ in self._consumers_store.values()] + [
        f for f, _, _ in self._producers_store.values()
    ]
    self.AppMocks = namedtuple(
        f"{self.__class__.__name__}Mocks", [f.__name__ for f in app_methods]
    )
    # todo: create Magicmock if needed
    self.mocks = self.AppMocks(**{f.__name__: AsyncMock() for f in app_methods})

    def add_mock(f: Callable[[...], Any], mock: AsyncMock) -> Callable[[...], Any]:
        @functools.wraps(f)
        async def async_inner(
            *args, f: Callable[[...], Any] = f, mock: AsyncMock = mock, **kwargs
        ) -> Any:
            await mock(*args, **kwargs)
            return await f(*args, **kwargs)

        return async_inner

    self._consumers_store = {
        name: (
            add_mock(f, getattr(self.mocks, f.__name__)),
            kwargs,
        )
        for name, (f, kwargs) in self._consumers_store.items()
    }
    self._producers_store = {
        name: (
            add_mock(f, getattr(self.mocks, f.__name__)),
            producer,
            kwargs,
        )
        for name, (f, producer, kwargs) in self._producers_store.items()
    }

In [ ]:
class TestMsg(BaseModel):
    msg: str = Field(...)

def create_service(bootstrap_servers: str) -> FastKafka:
    service = FastKafka(bootstrap_servers=bootstrap_servers)

    @service.consumes()
    async def on_preprocessed_signals(msg: TestMsg):
        await to_predictions(TestMsg(msg="prediction"))


    @service.produces()
    async def to_predictions(prediction: TestMsg) -> TestMsg:
        print(f"Sending prediction: {prediction}")
        return prediction
    
    return service

In [ ]:
service = create_service("localhost:9092")

service.create_mocks()
service.mocks.on_preprocessed_signals.assert_not_awaited()
service.mocks.to_predictions.assert_not_awaited()
service.create_mocks()
service.mocks.on_preprocessed_signals.assert_not_awaited()
service.mocks.to_predictions.assert_not_awaited()

In [ ]:
# | export

class Tester(FastKafka):
    def __init__(self, apps: List[FastKafka]):
        self.apps = apps
        super().__init__(bootstrap_servers=apps[0]._kafka_config["bootstrap_servers"])
        self.create_mirror()

        @self.produces()
        async def to_preprocessed_signals(msg: str) -> str:
            print(f"Producing msg {msg}")
            return msg
        self.to_preprocessed_signals = to_preprocessed_signals

        @self.consumes(auto_offset_reset="latest")
        async def on_predictions(msg: str):
            pass

    async def startup(self):
        for app in self.apps:
            app.create_mocks()
            await app.startup()
        
        self.create_mocks()
        await super().startup()
        await asyncio.sleep(3)

    async def shutdown(self):
        await super().shutdown()
        for app in self.apps.reverse():
            await app.shutdown()
        
    def create_mirror(self):
        pass

In [ ]:
tester = Tester([create_service("localhost:9092")])

with pytest.raises(AttributeError) as e:
    tester.mocks

In [ ]:
async with LocalKafkaBroker(zookeeper_port=9998, listener_port=9789) as bootstrap_server:
    create_missing_topics(
        ["preprocessed_signals", "predictions"],
        bootstrap_servers=bootstrap_server,
        num_partitions=1,
    )
    tester = Tester([create_service(bootstrap_servers=bootstrap_server)])
    async with tester:
        await tester.to_preprocessed_signals(TestMsg(msg="signal"))
        await asyncio.sleep(5)
        tester.mocks.on_predictions.assert_called()

print("ok")

[INFO] fastkafka.testing: Java is already installed.
[INFO] fastkafka.testing: But not exported to PATH, exporting...
[INFO] fastkafka.testing: Kafka is already installed.
[INFO] fastkafka.testing: But not exported to PATH, exporting...
[INFO] fastkafka.testing: Starting zookeeper...
[INFO] fastkafka.testing: Zookeeper started, sleeping for 5 seconds...
[INFO] fastkafka.testing: Starting Kafka broker...
[INFO] fastkafka.testing: Kafka broker started, sleeping for 5 seconds...
[INFO] fastkafka.testing: Local Kafka broker up and running on 127.0.0.1:9789
[INFO] fastkafka.helpers: create_missing_topics(['preprocessed_signals', 'predictions']): new_topics = [NewTopic(topic=preprocessed_signals,num_partitions=1), NewTopic(topic=predictions,num_partitions=1)]
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:9789'}'
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers':

[ERROR] aiokafka: Unable to update metadata from [0]
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 32620 terminated.
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 32254...
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka.consumer.fetcher: Failed fetch messages from 0: NodeNotReadyError: Attempt to send a request to node which is not ready (node id 0).
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka: Unable to update metadata from [0]
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka.consumer.fetcher: Failed fetch messages from 0: NodeNotReadyError: Attempt to send a request to node which is not ready (node id 0).
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call f

[ERROR] aiokafka.consumer.fetcher: Failed fetch messages from 0: NodeNotReadyError: Attempt to send a request to node which is not ready (node id 0).
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka: Unable to update metadata from [0]
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka.consumer.fetcher: Failed fetch messages from 0: NodeNotReadyError: Attempt to send a request to node which is not ready (node id 0).
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka: Unable to update metadata from [0]
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka.consumer.fetcher: Failed fetch messages from 0: NodeNotReadyError: Attempt to send a request to node which is not ready (node id 0).
[ERROR] aiokafka: Unable

NameError: name 'asyncio' is not defined

[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka.consumer.fetcher: Failed fetch messages from 0: NodeNotReadyError: Attempt to send a request to node which is not ready (node id 0).
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka.consumer.fetcher: Failed fetch messages from 0: NodeNotReadyError: Attempt to send a request to node which is not ready (node id 0).
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka: Unable to update metadata from [0]
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka: Unable to update metadata from [0]
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka.consumer.fetcher: Failed fetch messages from 0: Node

...

[ERROR] aiokafka: Unable to update metadata from [0]
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka.consumer.fetcher: Failed fetch messages from 0: NodeNotReadyError: Attempt to send a request to node which is not ready (node id 0).
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka: Unable to update metadata from [0]
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka.consumer.fetcher: Failed fetch messages from 0: NodeNotReadyError: Attempt to send a request to node which is not ready (node id 0).
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka: Unable to update metadata from [0]
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka

[ERROR] aiokafka: Unable to update metadata from [0]
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka.consumer.fetcher: Failed fetch messages from 0: NodeNotReadyError: Attempt to send a request to node which is not ready (node id 0).
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka: Unable to update metadata from [0]
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka.consumer.fetcher: Failed fetch messages from 0: NodeNotReadyError: Attempt to send a request to node which is not ready (node id 0).
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka: Unable to update metadata from [0]
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka

[ERROR] aiokafka: Unable to update metadata from [0]
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka.consumer.fetcher: Failed fetch messages from 0: NodeNotReadyError: Attempt to send a request to node which is not ready (node id 0).
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka: Unable to update metadata from [0]
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka.consumer.fetcher: Failed fetch messages from 0: NodeNotReadyError: Attempt to send a request to node which is not ready (node id 0).
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka: Unable to update metadata from [0]
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka

[ERROR] aiokafka: Unable to update metadata from [0]
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka.consumer.fetcher: Failed fetch messages from 0: NodeNotReadyError: Attempt to send a request to node which is not ready (node id 0).
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka: Unable to update metadata from [0]
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka.consumer.fetcher: Failed fetch messages from 0: NodeNotReadyError: Attempt to send a request to node which is not ready (node id 0).
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka: Unable to update metadata from [0]
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka

[ERROR] aiokafka: Unable to update metadata from [0]
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka.consumer.fetcher: Failed fetch messages from 0: NodeNotReadyError: Attempt to send a request to node which is not ready (node id 0).
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka: Unable to update metadata from [0]
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka.consumer.fetcher: Failed fetch messages from 0: NodeNotReadyError: Attempt to send a request to node which is not ready (node id 0).
[ERROR] aiokafka: Unable connect to node with id 0: [Errno 111] Connect call failed ('172.22.0.6', 9789)
[ERROR] aiokafka: Unable to update metadata from [0]
